<a href="https://colab.research.google.com/github/rllima/IF704/blob/main/entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time
import pandas as pd

print(tf.__version__)

2.6.0


In [87]:
!wget https://raw.githubusercontent.com/rllima/IF704/main/data/train.csv
!wget https://raw.githubusercontent.com/rllima/IF704/main/data/test.csv

--2021-08-18 00:45:57--  https://raw.githubusercontent.com/rllima/IF704/main/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 354161 (346K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 345.86K  --.-KB/s    in 0.04s   

2021-08-18 00:45:57 (8.46 MB/s) - ‘train.csv’ saved [354161/354161]

--2021-08-18 00:45:58--  https://raw.githubusercontent.com/rllima/IF704/main/data/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 568765 (555K) [text/plain]
Saving to: ‘test.csv’

test.c

In [84]:
from collections import defaultdict


In [89]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token
            if token.find('http://') == 0 or token.find('https://') == 0:
                token = '<URL>'
            if token[0] == '@':
                token = '<USR>'
            ######################################
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [ ]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    # Add special tokens (or tags) to the dictionaries.
    # The first special token must have index 0.
    
    # Mapping tok2idx should contain each token or tag only once. 
    # To do so, you should extract unique tokens/tags from the tokens_or_tags variable
    # and then index them (for example, you can add them into the list idx2tok
    # and for each token/tag save the index into tok2idx).

    for twt in tokens_or_tags:
        for tok in twt:
            idx2tok.append(tok)
    idx2tok = list(set(idx2tok))
    idx2tok = special_tokens + idx2tok
    for i, v in enumerate(idx2tok):
        tok2idx[v] = i
    
    return tok2idx, idx2tok

In [90]:
train_tokens, train_tags = read_data('train.csv')
test_tokens, test_tags = read_data('test.csv')

[['<USR>',
  '<USR>',
  'they',
  'will',
  'be',
  'all',
  'done',
  'by',
  'Sunday',
  'trust',
  'me',
  '*wink*'],
 ['Made',
  'it',
  'back',
  'home',
  'to',
  'GA',
  '.',
  'It',
  'sucks',
  'not',
  'to',
  'be',
  'at',
  'Disney',
  'world',
  ',',
  'but',
  'its',
  'good',
  'to',
  'be',
  'home',
  '.',
  'Time',
  'to',
  'start',
  'planning',
  'the',
  'next',
  'Disney',
  'World',
  'trip',
  '.'],
 ["'",
  'Breaking',
  'Dawn',
  "'",
  'Returns',
  'to',
  'Vancouver',
  'on',
  'January',
  '11th',
  '<URL>'],
 ['<USR>',
  'perhaps',
  ',',
  'but',
  'folks',
  'may',
  'find',
  'something',
  'in',
  'the',
  'gallery',
  'that',
  'is',
  'helpful',
  'in',
  'their',
  'day-to-day',
  'work',
  'as',
  'well',
  '.',
  'Even',
  'just',
  'to',
  'use',
  'it',
  '.'],
 ['<USR>', 'aye', 'been', 'tonight', '-', 'excellent'],
 ['RT',
  '<USR>',
  ':',
  'RT',
  'this',
  'if',
  'you',
  'want',
  'me',
  'to',
  'go',
  'back',
  'live',
  'on',
  'Ustr

In [77]:
train_examples[:10]

[[['@SammieLynnsMom',
   '@tg10781',
   'they',
   'will',
   'be',
   'all',
   'done',
   'by',
   'Sunday',
   'trust',
   'me',
   '*wink*',
   'Made',
   'it',
   'back',
   'home',
   'to',
   'GA',
   '.',
   'It',
   'sucks',
   'not',
   'to',
   'be',
   'at',
   'Disney',
   'world',
   ',',
   'but',
   'its',
   'good',
   'to',
   'be',
   'home',
   '.',
   'Time',
   'to',
   'start',
   'planning',
   'the',
   'next',
   'Disney',
   'World',
   'trip',
   '.',
   "'",
   'Breaking',
   'Dawn',
   "'",
   'Returns',
   'to',
   'Vancouver',
   'on',
   'January',
   '11th',
   'http://bit.ly/dbDMs8',
   '@ls_n',
   'perhaps',
   ',',
   'but',
   'folks',
   'may',
   'find',
   'something',
   'in',
   'the',
   'gallery',
   'that',
   'is',
   'helpful',
   'in',
   'their',
   'day-to-day',
   'work',
   'as',
   'well',
   '.',
   'Even',
   'just',
   'to',
   'use',
   'it',
   '.',
   '@Carr0t',
   'aye',
   'been',
   'tonight',
   '-',
   'excellent',
   'RT